In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
torch.cuda.is_available(), torch.backends.cudnn.enabled

(True, True)

In [4]:
PATH = "data/dogbreed/"
sz = 224
arch= resnext50
bs = 64

In [5]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)          # get random 20% of labels for val idx

## Step 1
### Enable data augmentation, precompute=True

In [6]:
# create function to make it easy to vary img sz and bs for sota technique
# of increasing img sz and bs to prevent overfitting
def get_data(sz, bs):
    tfms = tfms_from_model(arch,sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH,'train', f'{PATH}labels.csv',
                                        test_name='test', num_workers=4,
                                        val_idxs=val_idxs, suffix=".jpg",
                                        tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340,'tmp')   

In [7]:
data = get_data(sz, bs)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)
#learn.fit(1e-2, 5)

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

FileNotFoundError: [Errno 2] No such file or directory: '/home/snoopy/stuff/myfai/fastai/weights/resnext_50_32x4d.pth'

## Step 2
### Learning Rate Finder

In [ ]:
lfr = learn.lr_find()

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
1e-1, pow(10,-1), 10**-1

## Step 3
### Train last layer with precomputed activations

In [ ]:
learn.fit(1e-2, 2)

## Step 4
### Train last layer with data augmentation(ie. precompute=False) for 2-3 epochs, cycle_len=1

In [ ]:
learn.precompute=False
learn.fit(1e-2, 5, cycle_len=1)

## Step 5 jump to Step 8
### Step 5 - unfreeze all layers
### Step 6 - set differential lr
### step 7 - find lr again ( for what?)
### step 8 - Train full network with cycle_mult=2 until overfitting

### Did not unfreeze becos result is not getting better, because images are too similar to imagenet's, thus doesn't improve to train the filter layers. So we go straight to Step 8

In [ ]:
learn.set_data(get_data(299,bs))
learn.freeze()
% time learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

### File format for kaggle
'''
id,affenpinscher,afghan_hound,..,yorkshire_terrier

000621fb3cbb32d8935728e48679680e,0.0083,0.0,...,0.0083
etc.
'''

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),axis=0)
accuracy_np(probs, y)
#metrics.log_loss(y, probs)

In [ ]:
log_preds, y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds), axis=0)
accuracy_np(probs, y) #metrics.log_loss(y, probs)

In [ ]:
probs.shape

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
SUBM = f'{PATH}subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]

In [ ]:
fn

In [ ]:
Image.open((PATH+fn).resize(150,150))

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

im = trn_tfms(Image.open(PATH+fn))
preds = learn.predict_array(im[None]) # create a mini-batch of 1 image for learn.predict_array
np.argmax(preds)